In [79]:
import os
import string
import annoy
import json
import pickle
import re

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from gensim.models import Word2Vec

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import codecs

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model, metrics, model_selection, preprocessing

In [94]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

# Релизуем предобработку текста для классификатора
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("nan", "", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

# Релизуем предобработку текста для болталки и поиска товаров
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

#### Обучение классификатора «товарный запрос vs. болталка»

In [81]:
#Загрузим датасет с продуктами
data = pd.read_csv('ProductsDataset.csv')
data.info()
#Загрузим ответы из ранее обученной модели (болталки)
index_map = pickle.load(open("index_map",'rb'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35548 entries, 0 to 35547
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           35548 non-null  object 
 1   descrirption    33537 non-null  object 
 2   product_id      35536 non-null  object 
 3   category_id     35536 non-null  float64
 4   subcategory_id  35536 non-null  object 
 5   properties      35536 non-null  object 
 6   image_links     35533 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.9+ MB


In [82]:
# Объединим в один стобец название с описанием продукта и предобработаем
data['text']= data['title'].apply(preprocess_text) + ' ' + data['descrirption'].apply(preprocess_text)
# Целевая переменная для классификации
data['target'] = True

In [ ]:
# Дополним датасет данными из "болталки" и целевую переменную False 
talker = pd.DataFrame(list(index_map.items()),
                   columns=['index', 'text'])
talker = talker[talker.index < len(data)]
talker['target'] = False
talker = talker.drop(['index'], axis=1)
talker['text'] = talker['text'].apply(preprocess_text)
data_class = pd.concat([data, talker], ignore_index = True)

In [89]:
# делим выборку на обучающую и валидационную.
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data_class['text'], data_class['target'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [90]:
# Векторизируем текст при помощи TfidfVectorizer
tfidf_vec = TfidfVectorizer().fit(train_x.values)
xtrain_tfidf = tfidf_vec.transform(train_x)
xvalid_tfidf = tfidf_vec.transform(valid_x)

In [91]:
# Обучим классификатор методом LogisticRegression на на Word Level TF IDF Vectors
# Расчитаем метрику accuracy_score на валидации
classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_tfidf, train_y)
predictions = classifier.predict(xvalid_tfidf)
accuracy = metrics.accuracy_score(predictions, valid_y)
print("LR, WordLevel TF-IDF: ", accuracy)

LR, WordLevel TF-IDF:  0.9794081242263981


In [92]:
# Сохраняем модель классификатор
# pickle.dump(classifier, open("text_classifier",'wb'))
# Загружаем модель классификатор
classifier_model = pickle.load(open("text_classifier", 'rb'))

#### Реализуем поиск похожих товаров в контентной части бота

In [95]:
# Создадим список слов из названия и описания продуктов
titles = []

for line in tqdm_notebook(data.text):
    spls = preprocess_txt(line)
    titles.append(spls)

# Векторизируем список слов при помощи модели Word2Vec и сохраним модель 
titles = [i for i in titles if len(i) > 2]
product_model = Word2Vec(sentences=titles, vector_size=100, min_count=1, window=5)
product_model.save("w2v_product_model")

<ipython-input-95-fc723f8e14e7>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(data.text):


  0%|          | 0/35548 [00:00<?, ?it/s]

In [96]:
# Сложим в индексы все названия продуктов. Используем библиотеку annoy. 
# Проходимся по всем продуктам, считаем, что вектор продукта - усредненная сумма word2vecов слов.
product_index = annoy.AnnoyIndex(100 ,'angular')

index_map_titles = {}
counter = 0

for quest in data.title:
    n_w2v = 0
    index_map_titles[counter] = str(data.product_id[counter]) + ' ' + data.title[counter]
    question = preprocess_txt(quest)
    vector = np.zeros(100)
    for word in question:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    product_index.add_item(counter, vector)
        
    counter += 1

product_index.build(10)
#product_index.save('titles.ann')

True

In [97]:
# Функция для поиска продуктов
def find_product(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    query_index = product_index.get_nns_by_vector(vector, 1)
    return index_map_titles[query_index[0]]

#### Реализуем болталку

In [98]:
# Для болталки возьмем обученную ранее модель и созданные индексы
answer_model = Word2Vec.load("w2v_model")
answer_index = annoy.AnnoyIndex(100 ,'angular')
answer_index.load('speaker.ann')

True

In [99]:
# Функция для ответов на запросы "болталки"
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in answer_model.wv:
            vector += answer_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    query_index = answer_index.get_nns_by_vector(vector, 1)
    return index_map[query_index[0]]

#### Напишем функцию и проверим работу нашего чат-бота-«барахольщика»

In [ ]:
# Функция для ответа на запрос пользователя
def get_answer(answer):
    ans = pd.Series([answer], dtype="string")
    answer_tfidf = tfidf_vec.transform(ans) # Векторизируем запрос при помощи TfidfVectorizer
    if classifier.predict(answer_tfidf)==1: # Если запрос был продуктовым, возвращаем id и название продукта
        return find_product(answer)
    else:
        return find_answer(answer) # иначе отправляем запрос в болталку

In [115]:
# Применим автотесты к методу get_answer
assert(not get_answer("Где ключи от танка").startswith("5"))
assert(get_answer("Юбка детская ORBY").startswith("58e3cfe6132ca50e053f5f82"))

In [118]:
# Проверим наглядно два возможных сценария: продуктовый запрос и нет
get_answer('Новая золотая ветровка')

'5bd6c8fb2138bbc55745362c Новая золотая ветровка'

In [119]:
get_answer('Сколько весит слон?')

'3 тонны. \n'